In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn


In [ ]:
! pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.3 MB/s eta 0:00:00


In [2]:

# data = pd.read_csv("newfeature-.csv")
# data

df = pd.read_csv("label_deposit_nondeposit.csv")
df



,OBJECTID,pointid,grid_code,FuzzyMe_Eu,FuzzyMe_de,FuzzyMe_se,FuzzyMe_ar,FuzzyMe__1,FuzzyMe__2,x,y,class
0,1,38237,0.0,0.999985,0.400412,0.997949,1.000000,1.000000,1.000000,524700.0,3.231886e+06,1.0
1,2,38238,0.0,0.999995,0.320686,0.998934,1.000000,1.000000,1.000000,524900.0,3.231886e+06,1.0
2,3,38480,0.0,0.999986,0.350364,0.998249,1.000000,0.999998,1.000000,524700.0,3.231686e+06,1.0
3,4,38481,0.0,0.999996,0.343995,0.998768,1.000000,0.999999,1.000000,524900.0,3.231686e+06,1.0
4,5,38723,0.0,0.999986,0.461420,0.998934,1.000000,0.999989,1.000000,524700.0,3.231486e+06,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1482,758,1862,0.0,0.714838,0.000000,0.993068,0.375918,0.999648,0.999814,532700.0,3.261886e+06,0.0
1483,759,1863,0.0,0.740491,0.000000,0.995511,0.407588,0.999288,0.999655,532900.0,3.261886e+06,0.0
1484,760,1864,0.0,0.765215,0.000000,0.997166,0.440937,0.998815,0.999360,533100.0,3.261886e+06,0.0
1485,761,1865,0.0,0.788861,0.000000,0.998249,0.475781,0.999108,0.998949,533300.0,3.261886e+06,0.0


In [3]:
print(df.head())

   OBJECTID  pointid  grid_code  FuzzyMe_Eu  FuzzyMe_de  FuzzyMe_se  \
0         1    38237        0.0    0.999985    0.400412    0.997949   
1         2    38238        0.0    0.999995    0.320686    0.998934   
2         3    38480        0.0    0.999986    0.350364    0.998249   
3         4    38481        0.0    0.999996    0.343995    0.998768   
4         5    38723        0.0    0.999986    0.461420    0.998934   

   FuzzyMe_ar  FuzzyMe__1  FuzzyMe__2         x             y  class  
0         1.0    1.000000         1.0  524700.0  3.231886e+06    1.0  
1         1.0    1.000000         1.0  524900.0  3.231886e+06    1.0  
2         1.0    0.999998         1.0  524700.0  3.231686e+06    1.0  
3         1.0    0.999999         1.0  524900.0  3.231686e+06    1.0  
4         1.0    0.999989         1.0  524700.0  3.231486e+06    1.0  


In [4]:
from sklearn.preprocessing import StandardScaler

# Extract features and labels
features = df[['FuzzyMe_Eu', 'FuzzyMe_de', 'FuzzyMe_se', 'FuzzyMe_ar', 'FuzzyMe__1', 'FuzzyMe__2']].values
coordinates = df[['x', 'y']].values
labels = df['class'].values
labels = labels.astype(int)

# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(features)




In [5]:
from sklearn.model_selection import train_test_split
# (80% train, 20% val)
features_train, features_val, labels_train, labels_val, indices_train, indices_val = train_test_split(
    features, labels, df.index, test_size=0.2, random_state=42, stratify=labels)

In [6]:
from sklearn.neighbors import kneighbors_graph
# KNN graph for the training set
adjacency_matrix_train = kneighbors_graph(features_train, n_neighbors=2, mode='connectivity', include_self=True)
edge_index_train = np.array(adjacency_matrix_train.nonzero())

# KNN graph for the validation set
adjacency_matrix_val = kneighbors_graph(features_val, n_neighbors=2, mode='connectivity', include_self=True)
edge_index_val = np.array(adjacency_matrix_val.nonzero())


In [7]:
# Convert to PyTorch tensors for the training set
x_train = torch.tensor(features_train, dtype=torch.float)
y_train = torch.tensor(labels_train, dtype=torch.float).unsqueeze(1)  
edge_index_train = torch.tensor(edge_index_train, dtype=torch.long)

# Convert to PyTorch tensors for the validation set
x_val = torch.tensor(features_val, dtype=torch.float)
y_val = torch.tensor(labels_val, dtype=torch.float).unsqueeze(1)  
edge_index_val = torch.tensor(edge_index_val, dtype=torch.long)

In [10]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout):
        super(SimpleGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = dropout

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return torch.sigmoid(x)  

In [11]:

model = GCN(in_channels=features_train.shape[1], hidden_channels=64, out_channels=1, dropout=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCELoss() 

C:\ProgramData\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [ ]:
# Create PyTorch Geometric data objects for training and validation
data_train = Data(x=x_train, edge_index=edge_index_train, y=y_train)
data_val = Data(x=x_val, edge_index=edge_index_val, y=y_val)

In [ ]:
# Early stopping and learning rate scheduler
patience = 10
best_val_loss = float('inf')
patience_counter = 0
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

In [12]:
# Training loop with early stopping
model.train()
for epoch in range(300):
    optimizer.zero_grad()
    out = model(data_train)
    loss = loss_fn(out, data_train.y)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_output = model(data_val)
        val_loss = loss_fn(val_output, data_val.y)
        scheduler.step(val_loss)

    print(f'Epoch {epoch}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping")
        break


Epoch 0, Training Loss: 0.8166137337684631, Validation Loss: 0.704480767250061
Epoch 1, Training Loss: 0.7021862268447876, Validation Loss: 0.6339572072029114
Epoch 2, Training Loss: 0.62801194190979, Validation Loss: 0.5807798504829407
Epoch 3, Training Loss: 0.5721369385719299, Validation Loss: 0.5410828590393066
Epoch 4, Training Loss: 0.5300973653793335, Validation Loss: 0.5111111402511597
Epoch 5, Training Loss: 0.49832093715667725, Validation Loss: 0.4873180389404297
Epoch 6, Training Loss: 0.47300392389297485, Validation Loss: 0.4683487117290497
Epoch 7, Training Loss: 0.45267993211746216, Validation Loss: 0.45310038328170776
Epoch 8, Training Loss: 0.4361194372177124, Validation Loss: 0.44106897711753845
Epoch 9, Training Loss: 0.42274531722068787, Validation Loss: 0.4312674403190613
Epoch 10, Training Loss: 0.41155749559402466, Validation Loss: 0.42315950989723206
Epoch 11, Training Loss: 0.4019583463668823, Validation Loss: 0.41637954115867615
Epoch 12, Training Loss: 0.39366

Epoch 108, Training Loss: 0.22496531903743744, Validation Loss: 0.293935090303421
Epoch 109, Training Loss: 0.22398808598518372, Validation Loss: 0.29276975989341736
Epoch 110, Training Loss: 0.2230427861213684, Validation Loss: 0.2917023003101349
Epoch 111, Training Loss: 0.22209694981575012, Validation Loss: 0.29070112109184265
Epoch 112, Training Loss: 0.2211950570344925, Validation Loss: 0.28982457518577576
Epoch 113, Training Loss: 0.22038398683071136, Validation Loss: 0.28896287083625793
Epoch 114, Training Loss: 0.21962487697601318, Validation Loss: 0.2881377041339874
Epoch 115, Training Loss: 0.21887733042240143, Validation Loss: 0.28738489747047424
Epoch 116, Training Loss: 0.21813608705997467, Validation Loss: 0.2866860628128052
Epoch 117, Training Loss: 0.21740204095840454, Validation Loss: 0.2859306037425995
Epoch 118, Training Loss: 0.21667274832725525, Validation Loss: 0.28513848781585693
Epoch 119, Training Loss: 0.21593044698238373, Validation Loss: 0.28434133529663086


Epoch 211, Training Loss: 0.16250592470169067, Validation Loss: 0.23548643290996552
Epoch 212, Training Loss: 0.1621168553829193, Validation Loss: 0.23513533174991608
Epoch 213, Training Loss: 0.161734476685524, Validation Loss: 0.23483030498027802
Epoch 214, Training Loss: 0.1613481491804123, Validation Loss: 0.23462937772274017
Epoch 215, Training Loss: 0.16095499694347382, Validation Loss: 0.23439714312553406
Epoch 216, Training Loss: 0.16056379675865173, Validation Loss: 0.23405100405216217
Epoch 217, Training Loss: 0.16017375886440277, Validation Loss: 0.23374004662036896
Epoch 218, Training Loss: 0.15979768335819244, Validation Loss: 0.23361638188362122
Epoch 219, Training Loss: 0.15941354632377625, Validation Loss: 0.2333436757326126
Epoch 220, Training Loss: 0.1590244323015213, Validation Loss: 0.23299767076969147
Epoch 221, Training Loss: 0.1586470752954483, Validation Loss: 0.23264701664447784
Epoch 222, Training Loss: 0.15828509628772736, Validation Loss: 0.2324804961681366


In [13]:
# Evaluate the model on the validation set
model.eval()
with torch.no_grad():
    val_output = model(data_val)
    val_pred = (val_output > 0.5).float()  # Convert probabilities to binary predictions
    correct = (val_pred == data_val.y).sum()
    accuracy = int(correct) / len(data_val.y)
    print(f'Validation Accuracy: {accuracy:.4f}')

    train_output = model(data_train)
    train_pred = (train_output > 0.5).float()  # Convert probabilities to binary predictions
    correct = (train_pred == data_train.y).sum()
    accuracy = int(correct) / len(data_train.y)
    print(f'Train Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9128
Train Accuracy: 0.9462


In [ ]:
data2 = pd.read_csv("newfeature-.csv")

features = data2[['FuzzyMe_Eu', 'FuzzyMe_de', 'FuzzyMe_se', 'FuzzyMe_ar', 'FuzzyMe__1', 'FuzzyMe__2']].values
# labels = data2['class'].values

scaler = StandardScaler()
features = scaler.fit_transform(features)

# KNN graph for the training set
adjacency_matrix = kneighbors_graph(features, n_neighbors=2, mode='connectivity', include_self=False)
edge_index = np.array(adjacency_matrix.nonzero())


# Convert to PyTorch tensors for the training set
x = torch.tensor(features, dtype=torch.float)
# y_train = torch.tensor(labels_train, dtype=torch.long)
edge_index = torch.tensor(edge_index, dtype=torch.long)

# Create PyTorch Geometric data objects for training and validation
data = Data(x=x, edge_index=edge_index, y=None)


# Make predictions using the trained model
with torch.no_grad():
    predicted_labels = model(data)

unlabeled_data = data2.iloc[:, :]

# # Convert unlabeled_data to a DataFrame
unlabeled_data = pd.DataFrame(unlabeled_data)

# # Add predicted labels to the unlabeled data
unlabeled_data['Predicted_Labels'] = pd.Series(predicted_labels.view(-1).numpy(), index=unlabeled_data.index)

# # Save data to Excel file
unlabeled_data.to_excel('predicted_data_GCN_KNN2222.xlsx', index=False)
# predicted_labels





